In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import requests
from selenium import webdriver
import time 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from urllib.parse import urljoin
import csv

In [181]:
driver = webdriver.Chrome()
driver.get("https://www.admitsee.com/searchundergrad")


In [182]:
#Search 
inputElement = driver.find_element_by_id("combined_search1")
inputElement.send_keys("Harvard")


In [183]:
#click on the top search
driver.find_element_by_xpath("//li[@class='result']/a").click()

In [184]:
#click on Go 
driver.find_element_by_xpath("//input[@value='GO']").click()

In [190]:
#click on search result 
link = driver.find_elements_by_xpath("//div[@class='profile_card_button']/a")
[link.get_attribute("href") for link in link]

['https://www.admitsee.com/profile-detail/459999',
 'https://www.admitsee.com/profile-detail/373581',
 'https://www.admitsee.com/profile-detail/349777',
 'https://www.admitsee.com/profile-detail/337893',
 'https://www.admitsee.com/profile-detail/334915',
 'https://www.admitsee.com/profile-detail/328504',
 'https://www.admitsee.com/profile-detail/315122',
 'https://www.admitsee.com/profile-detail/257451',
 'https://www.admitsee.com/profile-detail/229381',
 'https://www.admitsee.com/profile-detail/229169',
 'https://www.admitsee.com/profile-detail/226559',
 'https://www.admitsee.com/profile-detail/191606',
 'https://www.admitsee.com/profile-detail/186496',
 'https://www.admitsee.com/profile-detail/185027',
 'https://www.admitsee.com/profile-detail/123173',
 'https://www.admitsee.com/profile-detail/88258',
 'https://www.admitsee.com/profile-detail/87383',
 'https://www.admitsee.com/profile-detail/70175',
 'https://www.admitsee.com/profile-detail/67352',
 'https://www.admitsee.com/profile-

In [215]:
def get_profile_urls(list_of_schools):   
    profile_urls = []
    for school in list_of_schools:
        print(f"Going through {school}".format(school))
        print(len(profile_urls))
        driver = webdriver.Chrome()
        driver.get("https://www.admitsee.com/searchundergrad")
        
        time.sleep(1)
        
        try:
            #Search
            inputElement = driver.find_element_by_id("combined_search1")
            inputElement.send_keys(school)

            time.sleep(3)

            #click on top search result 
            driver.find_element_by_xpath("//li[@class='result']/a").click()

            #click on Go 
            driver.find_element_by_xpath("//input[@value='GO']").click()
            
        except Exception as e:
            print(f"error in searching for...{school}".format(school))

    
        i = 1
        screen_height = driver.execute_script("return window.screen.height;")
        #scroll down
        while True:
            try:
                driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))

                i+=1

                time.sleep(3)

                scroll_height = driver.execute_script("return document.body.scrollHeight;")

                if (screen_height * i) > scroll_height:
                    temp = driver.page_source
                    break      

            except Exception as e:
                print(e)
                temp = driver.page_source
                break

        #after breaking read profile link and store
        try: 
            links = driver.find_elements_by_xpath("//div[@class='profile_card_button']/a")
            profile_urls +=[link.get_attribute("href") for link in links] 
        except Exceptionception as e: 
            print(f"problem retrieving links")
            
        driver.close()
        
    return profile_urls

        
    

In [226]:
def get_user_info(list_of_user_url):
    user_dictionary = {}
    
    with open('example_datafile.csv', 'w', newline='') as csvfile:
        fieldnames = ["user_url","username","current_school","user_major","hometown",
                     "gender","ethnicity","English_FL","Admission", "class_of","first_gen",
                     "accepted_schools","waitlisted_schools","rejected_schools","legacy","GPA_measure",
                     "GPA","rank","num_of_Essays", "num_of_Scores","num_of_Advice","num_of_ECs",
                     "num_of_Sports", "num_of_schools", "SAT", "ACT"]

        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
    
    for user_url in list_of_user_url:
        if user_url in user_dictionary: 
            continue
        else:
            try:
                user_dictionary[user_url] = None
                driver = webdriver.Chrome()     
                driver.get(user_url)

                time.sleep(2)

            except Exception as e: 
                print("failed to load user_url {user_url}".format(user_url))

            try:
                #URL
                user_url = user_url
                
                #username
                username = driver.find_element_by_xpath("//div[@class='user_name']").text

                #school
                current_school = driver.find_element_by_xpath("//div[@class='user_school']").text

                #major
                user_major = driver.find_element_by_xpath("//div[@class='user_major']").text

                hometown, gender, ethnicity, English_FL = None, None, None, None
                for i in range(len(driver.find_elements_by_xpath("//div[@class='hidethese']/span"))):
                    name = driver.find_elements_by_xpath("//div[@class='hidethese']/span")[i].get_attribute("innerHTML")
                
                    if name == "Hometown":
                        hometown = driver.find_elements_by_xpath("//div[@class='hidethese']/span")[i+1].get_attribute("innerHTML")

                    elif name == "Gender":
                        gender = driver.find_elements_by_xpath("//div[@class='hidethese']/span")[i+1].get_attribute("innerHTML")
                    elif name == "Ethnicity":
                        ethnicity = driver.find_elements_by_xpath("//div[@class='hidethese']/span")[i+1].get_attribute("innerHTML")

                    elif name == "English First Language":
                        English_FL  = driver.find_elements_by_xpath("//div[@class='hidethese']/span")[i+1].get_attribute("innerHTML")
                
                Admission, class_of, first_gen = None, None, None
                for i in range(len(driver.find_elements_by_xpath("//div[@class='p_panel_content']/span"))): 
                    name = driver.find_elements_by_xpath("//div[@class='p_panel_content']/span")[i].get_attribute("innerHTML")

                    if name == "Admission":
                        Admission = driver.find_elements_by_xpath("//div[@class='p_panel_content']/span")[i+1].get_attribute("innerHTML")
                    elif name == "Class of":
                        class_of = driver.find_elements_by_xpath("//div[@class='p_panel_content']/span")[i+1].get_attribute("innerHTML")

                    elif name == "First Gen College Student":
                        first_gen = driver.find_elements_by_xpath("//div[@class='p_panel_content']/span")[i+1].get_attribute("innerHTML")


                #dropdown for schools
                driver.find_element_by_xpath("//span[@class='tot']").click()

                #schools accepted to 
                accepted_schools = driver.find_element_by_xpath("//div[@class='enr acc sub summ']//div[@class='deet']").text

                #schools waitlisted to
                waitlisted_schools = driver.find_element_by_xpath("//div[@class='wai sub summ']//div[@class='deet']").text

                #schools rejected to
                rejected_schools = driver.find_element_by_xpath("//div[@class='rej sub summ']//div[@class='deet']").text

                #legacy 
                legacy = driver.find_element_by_xpath("//span[@class='dataschoollegacy']").text

                #GPA type
                GPA_measure = driver.find_elements_by_xpath("//div[@class='t_col']/div")[0].get_attribute("innerHTML")

                #GPA
                GPA = driver.find_elements_by_xpath("//div[@class='t_col']/div")[1].get_attribute("innerHTML")

                #Rank 
                rank = driver.find_elements_by_xpath("//div[@class='t_col']/div")[3].get_attribute("innerHTML")


                #STATs
                num_of_Essays, num_of_Scores, num_of_Advice, num_of_ECs, num_of_Sports =  None, None, None, None, None
                for i in range(len(driver.find_elements_by_xpath("//div[@class='stats']/div"))):
                    name = driver.find_elements_by_xpath("//div[@class='stats']/div")[i].text

                    if name == "Essays":
                        num_of_Essays = driver.find_elements_by_xpath("//div[@class='stats']")[i].text

                    elif name == "Scores":
                        num_of_Scores = driver.find_elements_by_xpath("//div[@class='stats']")[i].text

                    elif name == "Advice":
                        num_of_Advice = driver.find_elements_by_xpath("//div[@class='stats']")[i].text

                    elif name == "ECs":
                        num_of_ECs = driver.find_elements_by_xpath("//div[@class='stats']")[i].text

                    elif name == "Sports":
                        num_of_Sports = driver.find_elements_by_xpath("//div[@class='stats']")[i].text

                num_of_schools = driver.find_elements_by_xpath("//div[@class='stats totalschools']")[0].text



                #highschool type 
    #             highschool = driver.find_elements_by_xpath("//div[@class='p_content_padding']/span")[1].get_attribute("innerHTML")
    #             print(highschoolschool)
                
    

                SAT = driver.find_element_by_xpath("//div[@class='leftscore sc thesat2']").text            

                try:
                    ACT = driver.find_element_by_xpath("//div[@class='leftscore2 sc']").text
                except Exception as e: 
                    print("no ACT was found")
                    
                
                user_profile = {
                    
                    "user_url": user_url,
                    "username":username,
                    "current_school":current_school,
                    "user_major":user_major,
                    "hometown":hometown,
                    "gender":gender,
                    "ethnicity":ethnicity,
                    "English_FL":English_FL,
                    "Admission":Admission, 
                    "class_of":class_of,
                    "first_gen":first_gen,
                    "accepted_schools":accepted_schools,
                    "waitlisted_schools":waitlisted_schools,
                    "rejected_schools":rejected_schools,
                    "legacy":legacy,
                    "GPA_measure":GPA_measure,
                     "GPA":GPA,
                    "rank":rank,
                    "num_of_Essays":num_of_Essays_, 
                    "num_of_Scores":num_of_Scores,
                    "num_of_Advice":num_of_Advice,
                    "num_of_ECs":num_of_ECs,
                     "num_of_Sports":num_of_Sports,
                    "num_of_schools":num_of_schools,
                    "SAT":SAT,
                    "ACT":ACT
                    
                }
            
            writer.writerow(user_profile)

            except Exception as e: 
                print(f"Erroring Querying Data".format(user_url))

            driver.close()  
            
            
            

In [227]:
schools = ["Harvard", "Yale", "Penn State", "USC", "Duke", "MIT", "Princeton", "Stanford", "Columbia", "UPenn", 
          "UCLA", "University of Michigan", "Caltech", "Northwesten", "Chicago", "Brown", "Cornell", "Vanderbilt",
          "Berkeley", "Williams", "Dartmouth", "Amherst", "Rice", "Johns Hopkins", "Washington", "Ponoma", "Swarthmore",
          "CMU","Virginia", "Notre Dame", "Bowdoin", "UNC", "University Florida", "Georgetown", "Wellesley","Middlebury",
          "Georgia", "NYU", "Austin", "Emory", "Tuft", "Illinois", "Wisconsin", "Purdue", "Boston", "Arizona"]

In [217]:
urls = get_profile_urls(schools)

Going through Harvard
0
Going through Yale
310
Going through Penn State
552
Going through USC
720
Going through Duke
720
Going through MIT
1006
Going through Princeton
1008
Going through Stanford
1187
Going through Columbia
1493
Going through UPenn
1495
Going through UCLA
1992
Going through University of Michigan
2522
Going through Caltech
2893
Going through Northwesten
2931
error in searching for...Northwesten
Going through Chicago
8842
Going through Brown
8842
Going through Cornell
8842
Going through Vanderbilt
9216
Going through Berkeley
9427
Going through Williams
9429
Going through Dartmouth
9552
Going through Amherst
9747
Going through Rice
9838
Going through Johns Hopkins
9973
Going through Washington
10167
Going through Ponoma
10268
error in searching for...Ponoma
Going through Swarthmore
16179
Going through CMU
16247
Going through Virginia
16437
Going through Notre Dame
16439
Going through Bowdoin
16441
Going through UNC
16486
Going through University Florida
16487
error in se

In [221]:
urls